In [ ]:
import pandas as pd
import os
import keras

from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df = pd.read_csv('list_attribute.txt', delim_whitespace=True, skiprows=1)

file_images = os.listdir('images/')
df_filtered = df[df.index.isin(file_images)]

df_filtered.describe()

In [ ]:
df_filtered.reset_index(inplace=True)

df_compiled = df_filtered[['index', 'Male']]

df_compiled

In [ ]:
df_compiled[df_compiled['Male'] == -1] = 0
df_compiled['Male'] = df_compiled['Male'].astype('string')

In [ ]:
df_compiled.loc[df_compiled['Male'] == '1', ['Male']] = 'Male'
df_compiled.loc[df_compiled['Male'] == '0', ['Male']] = 'Female'
df_compiled

In [ ]:
train_files = os.listdir('train/')
test_files = os.listdir('test/')
df_train = df_compiled[df_compiled['index'].isin(train_files)]
df_test = df_compiled[df_compiled['index'].isin(test_files)]


In [ ]:
IMG_SIZE = (218, 178)
BATCH_SIZE = 30
EPOCH_COUNT = 20

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    './train',
    x_col='index',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    df_test,
    './test',
    x_col='index',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape= IMG_SIZE + (3,))

base_model.trainable = False
base_model.summary()

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam

adam = Adam(learning_rate=0.01)

In [ ]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=4,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.01)
# Define a ModelCheckpoint callback to save the model weights during training
checkpoint_callback = ModelCheckpoint(filepath='Dataset/vgg16_weights_lr01.h5',
                                      save_best_only=True,
                                      save_weights_only=False)

callbacks = [checkpoint_callback, earlystop, learning_rate_reduction]


In [ ]:

history = model.fit(train_generator,
                    epochs=EPOCH_COUNT,
                    validation_data=val_generator,
                    validation_steps=len(df_test)//BATCH_SIZE,
                    steps_per_epoch=len(df_train)//BATCH_SIZE,
                    verbose = 1,
                    callbacks=callbacks
                    )
